In [102]:
import torch
import numpy as np

from co3d.dataset.dataloader_zoo import dataloader_zoo
from co3d.dataset.dataset_zoo import dataset_zoo

In [27]:
category = "apple"
task = "singlesequence"
single_sequence_id = 0      # 0 또는 1

s_datasets = dataset_zoo(
        category=category,
        assert_single_seq=task == "singlesequence",
        dataset_name=f"co3d_{task}",
        test_on_train=False,
        load_point_clouds=False,
        test_restrict_sequence_id=single_sequence_id,
    )

set_names_mapping :{'train': ['test_known'], 'val': ['test_known', 'test_unseen'], 'test': ['test_known', 'test_unseen']}
???????
 ['189_20393_38136', '110_13051_23361'] 

Dataset :  train
subsets :  ['test_known']
Loading Co3D frames from ../dataset/CO3D/apple/frame_annotations.jgz.
Loading Co3D sequences from ../dataset/CO3D/apple/sequence_annotations.jgz.
Loading Co3D subset lists from ../dataset/CO3D/apple/set_lists.json.
Removing images with empty masks.
... filtered 36572 -> 35025
Limitting Co3D dataset to the '['test_known']' subsets.
pick_sequence: ['189_20393_38136']
... filtered 40 -> 1
Taking max 100 per sequence.
... filtered 81 -> 81
CO3D Dataset #frames=81
Dataset :  val
subsets :  ['test_known', 'test_unseen']
Loading Co3D frames from ../dataset/CO3D/apple/frame_annotations.jgz.
Loading Co3D sequences from ../dataset/CO3D/apple/sequence_annotations.jgz.
Loading Co3D subset lists from ../dataset/CO3D/apple/set_lists.json.
Removing images with empty masks.
... filtered 365

In [105]:
category = "apple"
task = "multisequence"

m_datasets = dataset_zoo(
        category=category,
        assert_single_seq=task == "singlesequence",
        dataset_name=f"co3d_{task}",
        test_on_train=False,
        load_point_clouds=False,
    )

set_names_mapping :{'train': ['train_known'], 'val': ['test_known', 'test_unseen', 'train_known', 'train_unseen'], 'test': ['test_known', 'test_unseen', 'train_known', 'train_unseen']}


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'list'

In [108]:
m_datasets["train"].__len__()       # train_known

24625

In [110]:
print(m_datasets["test"].__len__())     # train_known + train_unseen + test_known + test_unseen

print(m_datasets["test"].__len__() - m_datasets["train"].__len__())     # train_unseen + test_knwon + test_unseen

35025
10400


In [113]:
m_datasets["train"]

Co3dDataset(frame_annotations_file='../dataset/CO3D/apple/frame_annotations.jgz', sequence_annotations_file='../dataset/CO3D/apple/sequence_annotations.jgz', subset_lists_file='../dataset/CO3D/apple/set_lists.json', subsets=['train_known'], limit_to=-1, limit_sequences_to=-1, pick_sequence=[], exclude_sequence=[], limit_category_to=[], dataset_root='../dataset/CO3D', load_images=True, load_depths=True, load_depth_masks=True, load_masks=True, load_point_clouds=False, max_points=0, mask_images=False, mask_depths=False, image_height=800, image_width=800, box_crop=True, box_crop_mask_thr=0.4, box_crop_context=0.3, remove_empty_masks=True, n_frames_per_sequence=-1, seed=0, eval_batches=None)

In [132]:
batch_size = 5
batch_num_per_seq = 10

m_dataloaders = dataloader_zoo(
        m_datasets,
        batch_size=5,
        dataset_name=f"co3d_multisequence",
        images_per_seq_options=[batch_size * batch_num_per_seq],
    )

dataset_set : train, dataset : CO3D Dataset #frames=24625, batch_size : 5, dataset_len : 1000, dataset_len_val : 1, images_per_seq_options : [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
dataset_set : val, dataset : CO3D Dataset #frames=35025, batch_size : 5, dataset_len : 1000, dataset_len_val : 1, images_per_seq_options : [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
dataset_set : test, dataset : CO3D Dataset #frames=35025, batch_size : 5, dataset_len : 1000, dataset_len_val : 1, images_per_seq_options : [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [133]:
train_loader = m_dataloaders["train"]
val_loader = m_dataloaders["val"]
test_loader = m_dataloaders["test"]

In [134]:
len(train_loader)

1000

In [28]:
train_frame = next(iter(train_loader))
val_frame = next(iter(val_loader))
test_frame = next(iter(test_loader))

In [17]:
train_frame.image_rgb.shape

torch.Size([1, 3, 800, 800])

In [18]:
val_frame.image_rgb.shape

torch.Size([1, 3, 800, 800])

In [19]:
test_frame.image_rgb.shape

torch.Size([6, 3, 800, 800])

In [144]:
train_frame = next(iter(train_loader))

train_frame.image_path

['../dataset/CO3D/apple/110_13064_25256/images/frame000075.jpg',
 '../dataset/CO3D/apple/110_13064_25256/images/frame000047.jpg',
 '../dataset/CO3D/apple/110_13064_25256/images/frame000050.jpg',
 '../dataset/CO3D/apple/110_13064_25256/images/frame000056.jpg',
 '../dataset/CO3D/apple/110_13064_25256/images/frame000069.jpg']

In [95]:
# train_loader에서 리턴하는 아이템이 모두 동일한 시퀀스의 프레임인지 확인

for data_frame in tqdm(train_loader):
    image_paths = data_frame.image_path
    seq_names = [image_path.split('/')[4] for image_path in image_paths]

    seq_names_uniq = list(set(seq_names))
    if len(seq_names_uniq) != 1:
        print("다른 시퀀스의 프레임을 같이 리턴함...")
        break

  8%|▊         | 84/1000 [01:53<20:33,  1.35s/it]/mnt/srv/home/dlpc.1100/nerformer/co3d/dataset/scene_batch_sampler.py:68: UserWarning: Batch size smaller than self.batch_size! (This is fine for experiments with a single scene and viewpooling)
  warnings.warn(
100%|██████████| 1000/1000 [16:54<00:00,  1.01s/it]


In [101]:
seq_frames_num

{'45_2362_6728': 10,
 '220_23198_47985': 10,
 '308_32483_61126': 25,
 '41_1964_6048': 15,
 '39_1771_5181': 10,
 '198_21308_44513': 20,
 '220_23217_49295': 35,
 '363_38324_72981': 5,
 '45_2380_6820': 20,
 '41_1966_5872': 10,
 '210_22191_45413': 15,
 '339_35194_63840': 20,
 '39_1776_5186': 25,
 '43_2155_6526': 45,
 '239_25238_50942': 20,
 '47_2689_7493': 15,
 '198_21305_44480': 35,
 '353_37239_69784': 25,
 '43_2132_6344': 10,
 '363_38341_73276': 10,
 '43_2160_6537': 15,
 '39_1814_5254': 20,
 '350_36969_69347': 40,
 '308_32475_60385': 20,
 '239_25243_51256': 25,
 '161_17679_33270': 20,
 '210_22201_45657': 15,
 '354_37593_69712': 10,
 '339_35123_63240': 15,
 '351_37039_66934': 15,
 '43_2183_6564': 15,
 '353_37215_69638': 25,
 '229_24090_49263': 25,
 '272_29078_57798': 20,
 '272_29074_57381': 35,
 '210_22206_45753': 20,
 '272_29066_56725': 25,
 '12_108_643': 15,
 '43_2157_6531': 25,
 '210_22204_46641': 15,
 '39_1773_5183': 20,
 '41_1933_5579': 25,
 '341_35529_65357': 10,
 '41_1926_5864': 20

In [149]:
test_frame = next(iter(test_loader))

test_frame.image_path

['../dataset/CO3D/apple/220_23188_47084/images/frame000044.jpg',
 '../dataset/CO3D/apple/220_23188_47084/images/frame000053.jpg',
 '../dataset/CO3D/apple/220_23188_47084/images/frame000094.jpg',
 '../dataset/CO3D/apple/220_23188_47084/images/frame000061.jpg',
 '../dataset/CO3D/apple/220_23188_47084/images/frame000062.jpg',
 '../dataset/CO3D/apple/220_23188_47084/images/frame000058.jpg']

In [154]:
m_datasets["test"].__getitem__(2).image_path

'../dataset/CO3D/apple/12_90_489/images/frame000003.jpg'

In [86]:
val_frame = next(iter(val_loader))

val_frame.image_path

['../dataset/CO3D/apple/43_2208_6595/images/frame000064.jpg',
 '../dataset/CO3D/apple/43_2208_6595/images/frame000069.jpg',
 '../dataset/CO3D/apple/43_2208_6595/images/frame000083.jpg',
 '../dataset/CO3D/apple/43_2208_6595/images/frame000095.jpg',
 '../dataset/CO3D/apple/43_2208_6595/images/frame000061.jpg',
 '../dataset/CO3D/apple/151_16774_32223/images/frame000005.jpg',
 '../dataset/CO3D/apple/151_16774_32223/images/frame000060.jpg',
 '../dataset/CO3D/apple/151_16774_32223/images/frame000028.jpg',
 '../dataset/CO3D/apple/151_16774_32223/images/frame000087.jpg',
 '../dataset/CO3D/apple/151_16774_32223/images/frame000001.jpg']

In [5]:
import json

In [64]:
set_lists_file = "../dataset/CO3D/apple/set_lists.json"
single_batch_file = "../dataset/CO3D/apple/eval_batches_singlesequence.json"
multi_batch_file = "../dataset/CO3D/apple/eval_batches_multisequence.json"

with open(set_lists_file, 'r') as f:
    set_lists = json.load(f)

with open(single_batch_file, 'r') as f:
    single_batch = json.load(f)

with open(multi_batch_file, 'r') as f:
    multi_batch = json.load(f)

In [65]:
print(type(set_lists), set_lists.keys())

print(type(multi_batch), len(multi_batch))

print(type(single_batch), len(single_batch))

<class 'dict'> dict_keys(['train_known', 'train_unseen', 'test_known', 'test_unseen'])
<class 'list'> 1000
<class 'list'> 200


In [66]:
print(type(single_batch), type(multi_batch))

<class 'list'> <class 'list'>


In [39]:
from tqdm import tqdm

In [67]:
train_known_seq_names = []
for item in set_lists['train_known']:
    seq_name = item[0]
    train_known_seq_names.append(seq_name)
print(len(train_known_seq_names))
train_known_seq_names_uniq = list(set(train_known_seq_names))
print(len(train_known_seq_names_uniq))

train_unseen_seq_names = []
for item in set_lists['train_unseen']:
    seq_name = item[0]
    train_unseen_seq_names.append(seq_name)
print(len(train_unseen_seq_names))
train_unseen_seq_names_uniq = list(set(train_unseen_seq_names))
print(len(train_unseen_seq_names_uniq))


for train_unseen_seq_name in tqdm(train_unseen_seq_names_uniq):
    if train_unseen_seq_name not in train_known_seq_names_uniq:
        print("known에는 없는 시퀀스가 unseen에 있음")
        break

25841
356
6692
356


100%|██████████| 356/356 [00:00<00:00, 127523.46it/s]


In [68]:
sorted(train_unseen_seq_names_uniq) == sorted(train_known_seq_names_uniq)

True

In [69]:
train_known_seq_names = []
for item in set_lists['train_known']:
    seq_name = item[0]
    train_known_seq_names.append(seq_name)
print(len(train_known_seq_names))
train_known_seq_names_uniq = list(set(train_known_seq_names))
print(len(train_known_seq_names_uniq))

test_known_seq_names = []
for item in set_lists['test_known']:
    seq_name = item[0]
    test_known_seq_names.append(seq_name)
print(len(test_known_seq_names))
test_known_seq_names_uniq = list(set(test_known_seq_names))
print(len(test_known_seq_names_uniq))


for test_known_seq_name in tqdm(test_known_seq_names_uniq):
    if test_known_seq_name in train_known_seq_names_uniq:
        print("test에 있는 시퀀스가 train에도 있음")
        break

25841
356
3207
40


100%|██████████| 40/40 [00:00<00:00, 49622.05it/s]


In [70]:
print(sorted(train_known_seq_names_uniq)[:10])

['103_12148_21353', '110_13052_23173', '110_13053_23637', '110_13054_23182', '110_13057_23420', '110_13060_23672', '110_13064_25256', '120_14059_28189', '120_14065_29048', '120_14067_29180']


In [71]:
print(sorted(test_known_seq_names_uniq)[:10])

['110_13051_23361', '12_104_640', '151_16771_32072', '189_20388_37651', '189_20393_38136', '198_21279_40528', '198_21290_41362', '198_21306_44492', '210_22202_46172', '210_22216_46396']


In [102]:
print(single_batch[0])
print()
print(single_batch[100])

[['189_20393_38136', 92, 'apple/189_20393_38136/images/frame000093.jpg'], ['189_20393_38136', 100, 'apple/189_20393_38136/images/frame000101.jpg'], ['189_20393_38136', 34, 'apple/189_20393_38136/images/frame000035.jpg'], ['189_20393_38136', 61, 'apple/189_20393_38136/images/frame000062.jpg'], ['189_20393_38136', 32, 'apple/189_20393_38136/images/frame000033.jpg'], ['189_20393_38136', 11, 'apple/189_20393_38136/images/frame000012.jpg'], ['189_20393_38136', 78, 'apple/189_20393_38136/images/frame000079.jpg'], ['189_20393_38136', 15, 'apple/189_20393_38136/images/frame000016.jpg'], ['189_20393_38136', 19, 'apple/189_20393_38136/images/frame000020.jpg'], ['189_20393_38136', 65, 'apple/189_20393_38136/images/frame000066.jpg']]

[['110_13051_23361', 26, 'apple/110_13051_23361/images/frame000027.jpg'], ['110_13051_23361', 40, 'apple/110_13051_23361/images/frame000041.jpg'], ['110_13051_23361', 18, 'apple/110_13051_23361/images/frame000019.jpg'], ['110_13051_23361', 42, 'apple/110_13051_23361/

In [104]:
print("189_20393_38136" in test_known_seq_names_uniq)
print("110_13051_23361" in test_known_seq_names_uniq)

True
True


In [99]:
multi_batch[0]

multi_batch_seq_names = []
for multi_batch_seq_item in multi_batch:
    for item in multi_batch_seq_item:
        seq_name = item[0]
        multi_batch_seq_names.append(seq_name)
print(len(multi_batch_seq_names))
multi_batch_seq_names_uniq = list(set(multi_batch_seq_names))
print(len(multi_batch_seq_names_uniq))


for multi_batch_seq_name in tqdm(multi_batch_seq_names_uniq):
    if multi_batch_seq_name not in train_known_seq_names_uniq:
        print("배치 시퀀스 중에 train 시퀀스에 없는 것도 있음")
        break

5986
140


  4%|▍         | 6/140 [00:00<00:00, 8981.38it/s]

배치 시퀀스 중에 train 시퀀스에 없는 것도 있음


In [106]:
print(multi_batch_seq_names_uniq[5] in train_known_seq_names_uniq)
print(multi_batch_seq_names_uniq[6] in train_known_seq_names_uniq)

True
False
